# ПЛАН ИССЛЕДОВАНИЯ
**Тема:** Сравнение подходов классического машинного обучения (ML) и нейронных сетей (ANN) для вычисления градиента давления в двухфазных потоках.

---

## 1. Отбор данных и предобработка (Data Preparation)
**Цель:** Формирование качественного набора данных (dataset) и приведение экспериментальных параметров к критериям подобия.

### 1.1. Сбор данных (Data Mining)
* **Источники данных:** Извлечение экспериментальных точек из научных статей (базы данных по хладагентам R1234yf, R134a, R32 и CO2).
* **Целевая переменная (Target):**
    * Градиент давления $dP/dz$ ($kPa \cdot m^{-1}$).
* **Входные физические параметры:**
    * Массовый поток $G$ ($kg \cdot m^{-2} \cdot s^{-1}$).
    * Тепловой поток $\dot{q}$ ($kW \cdot m^{-2}$).
    * Паросодержание $x$ (диапазон $0 \dots 1$).
    * Температура насыщения $T_{sat}$ ($^\circ C$).
    * Внутренний диаметр трубки $D$ ($mm$).
    * Свойства жидкости и пара: плотность $\rho_l, \rho_v$; вязкость $\mu_l, \mu_v$; поверхностное натяжение $\sigma$.

### 1.2. Нормализация и обезразмеривание (Feature Engineering)
Преобразование физических величин в безразмерные числа для обеспечения универсальности модели.

* **Инерционные и вязкостные силы (Число Рейнольдса):**
  $$Re_l, Re_v, Re_{lo}, Re_{vo}$$
* **Инерционные силы и поверхностное натяжение (Число Вебера):**
  $$We_l, We_v, We_{lo}, We_{vo}$$
* **Гравитационные силы (Число Фруда):**
  $$Fr_l, Fr_v, Fr_{lo}, Fr_{vo}$$
* **Влияние поверхностного натяжения и гравитации (Число Бонда):**
  $$Bo$$
* **Параметр Локхарта-Мартинелли:**
  $$X$$
* **Отношения свойств фаз:**
  $$\rho_l / \rho_v, \quad \mu_l / \mu_v$$

**Процедура:**
1.  Расчет указанных чисел для каждой экспериментальной точки.
2.  Нормализация данных (Z-score normalization):
    $$z = \frac{x - \mu}{\sigma}$$
    где $\mu$ — среднее значение, $\sigma$ — стандартное отклонение.

---

## 2. Выбор моделей и архитектур (Model Selection)

### 2.1. Классическое машинное обучение (ML)
Выбор алгоритмов регрессии, способных улавливать нелинейные зависимости табличных данных:
* **Random Forest Regressor (RF):** Для оценки важности признаков (feature importance) безразмерных чисел.
* **Gradient Boosting (XGBoost / CatBoost):** Как основной конкурент нейросетям на табличных данных.
* **Support Vector Regression (SVR):** Для сравнения на малых выборках.

### 2.2. Нейронные сети (ANN)
Разработка нескольких архитектур для сравнения:

* **Модель A (Pure ANN):** Полносвязная сеть (Feed-forward), принимающая на вход вектор безразмерных чисел:
  $$Inputs = [Re, We, Fr, Bo, X, \dots]$$

* **Модель B (Hybrid C-RNN):** Гибридная модель «Корреляция + Остаточная нейросеть».
    * *Принцип:*
      $$dP/dz_{\text{final}} = dP/dz_{\text{corr}} + R_{\text{ANN}}$$
    * *Описание:* Использование физической корреляции (например, Kim and Mudawar) как базы, сеть предсказывает только ошибку (остаток $R$).

* **Архитектура:**
    * *Входной слой:* 10–12 нейронов (по числу значимых параметров).
    * *Скрытые слои:* Структура $[12, 8, 3]$ или $[12, 10, 8]$.
    * *Функция активации:* $\text{ReLU}$.

---

## 3. Методология анализа (Evaluation Framework)

### 3.1. Функции потерь (для обучения)
* **Mean Squared Error (MSE):** Основная функция потерь для обучения сетей.
  $$MSE = \frac{1}{N} \sum_{i=1}^{N} (y_{\text{pred}, i} - y_{\text{exp}, i})^2$$
* **Регуляризация:** $L2$ регуляризация ($\lambda = 0.001$) для предотвращения переобучения.

### 3.2. Метрики качества (для валидации)
Для сравнения точности с литературными данными:
* **Mean Percentage Absolute Error (MPAE):**
  $$MPAE = \frac{1}{N} \sum_{i=1}^{N} \left| \frac{y_{\text{pred}, i} - y_{\text{exp}, i}}{y_{\text{exp}, i}} \right| \times 100\%$$
* **Mean Percentage Error (MPE):** Для оценки систематического смещения (bias).
  $$MPE = \frac{1}{N} \sum_{i=1}^{N} \frac{y_{\text{pred}, i} - y_{\text{exp}, i}}{y_{\text{exp}, i}} \times 100\%$$

---

## 4. Этап реализации ML алгоритмов
1.  Разделение данных: Обучающая (70%), Валидационная (15%), Тестовая (15%) выборки с фиксированным `random seed`.
2.  Обучение моделей (RF, SVR, Boosting).
3.  **Анализ корреляций остатков:** Вычисление корреляции Пирсона между ошибкой модели ($R$) и безразмерными числами, чтобы определить слабые места модели.

---

## 5. Этап реализации Нейронных сетей
1.  **Обучение:** Использование оптимизатора Adam ($\beta_1=0.9, \beta_2=0.999$, learning rate $= 0.001$).
2.  **Early Stopping:** Остановка обучения, если Loss на валидации не падает в течение заданного количества эпох или достигнут лимит (например, 1000 эпох).
3.  **Сравнение архитектур:** Построение графика зависимости MSE от количества обучаемых параметров (Pareto front) для выбора оптимальной сложности сети.

---

## 6. Итоговое сравнение и валидация
1.  **Прямой математический расчет:**
    Сравнение прогнозов ИИ с классическими корреляциями:
    * Kim and Mudawar.
    * Muller-Steinhagen and Heck.
    * Cheng et al. (для CO2).
2.  **Сводная таблица:** Сравнение MPAE и MPE для:
    * Лучшей ML модели.
    * Чистой ANN.
    * Гибридной C-RNN.
    * Эмпирических корреляций.
3.  **Анализ экстраполяции:** Проверка работы моделей на внешних данных, не участвовавших в обучении, для оценки переобучения.